# Projeto - Detector de _tweets_ Antivacina - Análise Exploatória de Dados
Detector de tweets antivacina | Esse projeto tem como objetivo raspar tweets recentes da API oficial do Twitter, fazer uma análise exploratória das métricas e treinar e classificar automaticamente os tweets como potenciais pró-vacina ou antivacina. 

Projeto realizado como trabalho final da disciplina Pensamento Computacional, do programa de pós-graduação em Jornalismo de Dados, Automação e Data Storytelling do Insper

Profs: Guilherme Dias Felitti e Alvaro Jusen

OBS: Requisições realizadas localmente por problemas de RAM do Colab

In [ ]:
# instalando bibliotecas
! pip install altair
! pip install missingno
! pip install wordcloud

In [101]:
# importando bibliotecas para AED
import pandas as pd
import altair as alt  # visualização de dados
import missingno as msno  # verificação de missing data
import datetime
from ast import literal_eval

# importando bibliotecas necessárias para o wordcloud
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Etapa 6: Realizando Análise Exploratória de Dados

A análise irá levar em consideração o algoritmo "MaxentClassifier", pois apresentou um resultado mais satisfatório que os outros algoritmos

In [102]:
df_classificacao = pd.read_csv(r'tweets_classificados.csv', converters={'sem_stop': literal_eval})

## Apresentando gráfico de missing data no banco

In [ ]:
msno.matrix(df_classificacao)

## Demonstrando estatística descritiva para as métricas dos tweets

In [104]:
# Resumo estatístico - retweets
df_classificacao['retweet'].describe()

count   15255.00
mean     1335.85
std      1397.35
min         0.00
25%         2.00
50%       757.00
75%      2623.00
max     10143.00
Name: retweet, dtype: float64

In [105]:
# Resumo estatístico - likes
df_classificacao['likes'].describe()

count   15255.00
mean        1.31
std        58.62
min         0.00
25%         0.00
50%         0.00
75%         0.00
max      6554.00
Name: likes, dtype: float64

In [106]:
# Resumo estatístico - respostas
df_classificacao['respostas'].describe()  # aparentemente não há muitas respostas no banco de dados

count   15255.00
mean        0.11
std         1.92
min         0.00
25%         0.00
50%         0.00
75%         0.00
max       199.00
Name: respostas, dtype: float64

In [107]:
# Resumo estatístico - menções
df_classificacao['menções'].describe()  # aparentemente não há muitas menções no banco de dados

count   15255.00
mean        0.02
std         0.73
min         0.00
25%         0.00
50%         0.00
75%         0.00
max        85.00
Name: menções, dtype: float64

## Fazendo algumas análises

In [108]:
# tirando a notação científica
pd.set_option('display.float_format', lambda x: '%.2f' %x)

# agrupando tweets por tipo de análise
df_classificacao['id_autor'] = df_classificacao['id_autor'].map(str)
agrupado_classificacao = df_classificacao.groupby(['classificacao_MaxentClassifier']).describe()
agrupado_classificacao.head()

Unnamed: 0                               \
                                    count    mean     std  min     25%   
classificacao_MaxentClassifier                                           
antivacina                        2347.00 8906.31 4518.14 1.00 5314.50   
neutro                            6146.00 7233.17 4812.72 6.00 2943.75   
pro_vacina                        6762.00 7759.10 3973.65 0.00 4838.50   

                                                          retweet          \
                                    50%      75%      max   count    mean   
classificacao_MaxentClassifier                                              
antivacina                     10085.00 12880.50 15415.00 2347.00  129.88   
neutro                          5927.50 11926.25 17296.00 6146.00 2055.53   
pro_vacina                      7809.00 10547.50 15417.00 6762.00 1100.29   

                                ... likes         menções                      \
                                ...   75%     max   count mean  std  min  25%   
classificacao_MaxentClassifier  ...                                             
antivacina                      ...  0.00 1055.00 2347.00 0.02 0.20 0.00 0.00   
neutro                          ...  0.00 6554.00 6146.00 0.02 1.09 0.00 0.00   
pro_vacina                      ...  0.00 2449.00 6762.00 0.01 0.32 0.00 0.00   

                                                
                                50%  75%   max  
classificacao_MaxentClassifier                  
antivacina                     0.00 0.00  7.00  
neutro                         0.00 0.00 85.00  
pro_vacina                     0.00 0.00 18.00  

[3 rows x 40 columns]

In [109]:
# agrupando tweets por tipo de análise
somado_classificacao = df_classificacao.groupby(['classificacao_MaxentClassifier']).sum()


# Criando função com resposta da análise automática
def rank_metrica(metrica):
    metrica_desejada = somado_classificacao.sort_values(by=metrica, ascending=False)
    primeiro = metrica_desejada[metrica].keys()[0]
    valor = metrica_desejada[metrica][0]
    print(f'O tipo de classificação que tem mais {metrica} é o {primeiro}, com a contagem de {valor} {metrica}')

In [110]:
# mostrando saida de texto automática para métricas somadas
metricas = ['retweet', 'likes', 'respostas', 'menções']

for metrica in metricas:
    rank_metrica(metrica)

O tipo de classificação que tem mais retweet é o neutro, com a contagem de 12633304 retweet
O tipo de classificação que tem mais likes é o neutro, com a contagem de 9609 likes
O tipo de classificação que tem mais respostas é o pro_vacina, com a contagem de 681 respostas
O tipo de classificação que tem mais menções é o neutro, com a contagem de 119 menções


# Etapa 7: Tratando o dataframe usando ```pd.melt()``` e Visualizando dados

Criando um dataframe com tidy data para tornar possível visualizações melhores

In [111]:
# Deixando o dado em tidy - problema com as métricas
df_tidy = pd.melt(
    df_classificacao,
    id_vars = ['tweet', 'fonte', 'data_postagem', 'idioma', 'id_autor', 'tipo', 'bag_of_words', 'classificacao_MaxentClassifier', 'classificacao_SklearnClassifier', 'classificacao_NaiveBayesClassifier', 'sem_stop'],
    var_name = 'tipo_metrica',
    value_name= 'valor_metrica'
)

In [112]:
# Transformando datetime em strings por causa de um bug do altair (https://github.com/altair-viz/altair/issues/1931)
df_tidy['data_postagem'] = df_tidy['data_postagem'].map(str)

In [113]:
# Retirando métricas zeradas
df_tidy = df_tidy.query('  tipo_metrica != "Unnamed: 0"  ')

In [ ]:
# desativando o max_row do altair
alt.data_transformers.disable_max_rows()

# verificando outliers com boxplot
alt.Chart(df_tidy).mark_boxplot().encode(
    x = 'tipo_metrica',
    y = 'valor_metrica',
    color = 'tipo',
    tooltip = ['tweet']).interactive()

In [ ]:
# verificando outliers com boxplot
alt.Chart(df_tidy).mark_boxplot().encode(
    x = alt.X('classificacao_MaxentClassifier', title = None),
    y = 'valor_metrica',
    column = 'tipo_metrica',
    color = 'tipo',
    tooltip = ['tweet']).interactive()

In [1]:
# fazendo função de geração de nuvem de palavras
def nuvem_palavras(dataframe):
   
    tuite = []
    for indice, coluna in dataframe.iterrows():
        dataframe_limpo = ' '.join(coluna['sem_stop'])
        tuite.append(dataframe_limpo)

    # motando nuvem de palavras geral do dataframe
    nuvem = WordCloud().generate(str(tuite))
    plt.imshow(nuvem, interpolation='bilinear')
    plt.axis('off')

In [ ]:
# Fazendo nuvem de palavras geral
nuvem_palavras(df_tidy)

In [ ]:
# fazendo somente para tweets classificados como pró-vacina
pro_vacina_tweets = df_tidy.query('  classificacao_MaxentClassifier == "pro_vacina"   ')
nuvem_palavras(pro_vacina_tweets)

In [ ]:
# fazendo somente para tweets classificados como antivacina
antivacina_tweets = df_tidy.query('  classificacao_MaxentClassifier == "antivacina"   ')
nuvem_palavras(antivacina_tweets)